In [1]:
import requests
import re
import numpy as np
import time
from bs4 import BeautifulSoup
import pandas as pd
import json 
import requests
import datetime
from datetime import date
import warnings
warnings.filterwarnings("ignore")
pd.set_option('max_colwidth', 800)

In [2]:
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key='a539d7df2c7b43e1ac4d12f386d901e8')

In [3]:
x = newsapi.get_top_headlines(country='us', page_size=50, category = 'health')

In [4]:
nyt_api_key = '13bd501bc77542a58e2e6678619b0d60'

In [5]:
today = int(str(date.today()).replace('-',''))
last_week = int(str(date.today() - datetime.timedelta(days = 14)).replace('-',''))

# NYT & NewsAPI

In [18]:
import requests
import re
import numpy as np
import time
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
import datetime
from datetime import date
from apikeys import *
from info import *

#dates to use for API call
today = int(str(date.today()).replace('-',''))
last_week = int(str(date.today() - datetime.timedelta(days = 14)).replace('-',''))

#NEW YORK TIMES
#clean the response from NYT API
def NYT_title_clean(df):
    titles = []
    for index, row in df.iterrows():
        title = row.headline['main']
        titles.append(title)
    df['title'] = titles
    return df

def NYT_dropped_rows(df):
    df.pub_date = pd.to_datetime(df.pub_date).dt.date
    df.word_count = round(df.word_count / 150)
    df.document_type = 'text'
    df['formality'] = 'Intermediate'
    return df

def NYT_dataframe_clean(df):
    dataframe = NYT_title_clean(df)
    dataframe = NYT_dropped_rows(dataframe)
    return dataframe

def NYT_api_call_section_based(section, source, page, start, end, key):
    url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:({section_name})&page={page}&source:({source})&begin_date={start}&end_date={end}&api-key={api}'.format(section_name = section, page = page, source = source, start = start, end = end, api = key)
    resp = requests.get(url=url)
    data = json.loads(resp.text)
    df = pd.DataFrame(data['response']['docs'])
    df = NYT_dataframe_clean(df)
    df['param'] = section
    df['image_url'] = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSHEtiVXw8Wi1tp56Nzd5rH_EoOAJA2RInEWvf5h5CQ-6O_YZp7dw'
    return df

def NYT_api_call_parameter_ALLTIME(param, page, key):
    url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?q={param}&page={page}&sort=newest&&api-key={api}'.format(param = param, page = page, api = key)
    resp = requests.get(url=url)
    data = json.loads(resp.text)
    df = pd.DataFrame(data['response']['docs'])
    df = NYT_dataframe_clean(df)
    df['param'] = param
    df['image_url'] = 'https://greaterbostonhcs.com/wp-content/uploads/2016/05/Nutrition.jpg'
    return df

def NYT_pull(categories):
    empty = pd.DataFrame()
    for word in categories:
        try:
            df = NYT_api_call_parameter_ALLTIME(word,0,nyt_api_key)
            empty = empty.append(df, sort=True)
            print('Pulled '+word)
            time.sleep(2)
        except:
            print(word + " EXCEPTION!!!!")
    empty = empty.drop(['abstract','section_name'],axis = 1)
    empty = empty.rename(index=str, columns={"_id": "source_id", "document_type": "medium",'pub_date':'date','snippet':'description','word_count':'length'})
    return empty

#NEWSAPI

def rename_columns(df):
    df = df.rename(index=str, columns={'publishedAt':'date','url':'web_url','urlToImage':'image_url'})
    return df

def add_words(df):
    lengths = []
    for string in df.content:
        try:
            lengths.append(round(int(string[string.find('+')+1:string.find(' chars')]) / 4 / 250))
        except:
            lengths.append(4)
    return lengths

def split_source_info(list_of_dicts):
    for item in list_of_dicts:
        item['source_id'] = item['source']['id']
        item['source'] = item['source']['name']

def pull_articles(parameter):
    article_results_rel = newsapi.get_everything(q=parameter,sort_by = 'relevancy',language='en', page_size=10, sources=sources_joined)
    article_results_rel = article_results_rel['articles']
    split_source_info(article_results_rel)
    return article_results_rel

def clean_articles(list_of_dicts, search_param):
    df = pd.DataFrame(list_of_dicts)
    try:
        df['medium'] = 'text'
        df['param'] = search_param
        df['publishedAt'] = df['publishedAt'].apply(lambda x: pd.to_datetime(x).date().strftime('%Y-%m-%d'))
        df['formality'] = 'Intermediate'
        df['length'] = add_words(df)
        df = rename_columns(df)
        print(search_param)
    except:
        pass
    return df


def call_news_api(categories):
    empty_df = pd.DataFrame()
    for category in categories:
        dicts = pull_articles(category)
        df = clean_articles(dicts, category)
        try:
            empty_df = empty_df.append(df, sort=True)
        except:
            pass
        print(len(empty_df.index))
    return empty_df


# Social Media

In [214]:
import tweepy
from apikeys import *
import pandas as pd
import json
import requests
import datetime
from datetime import date
from info import *

auth = tweepy.OAuthHandler(twitter_1, twitter_2)
auth.set_access_token(twitter_3, twitter_4)
api = tweepy.API(auth)
#all twitter handles to scrape
# twitter_handles = ['@ATPScience1', '@waitrose', '@MicrobiomeInst', '@veganrecipescom', '@cldiet', '@Onnit', '@vegsoc', '@VeganKosher', '@TheVeganSociety', '@vegan', '@Keto_Recipes_', '@the52diet', '@IFdiet', '@microbiome', '@metagenomics', '@microbiome_news', '@TheGutStuff', '@MyGutHealth', '@PaleoFX',
# '@PaleoFoundation', '@ThePaleoDiet', '@PaleoComfort', '@cavemanketo', '@KetoFlu', '@TheKetoKitchen_', '@EatKetoWithMe', '@KetoConnect', '@KetoDietZone', '@Ketogenic', '@USDANutrition', '@FoodRev', '@CSPI', '@simplyrecipes', '@FoodNetwork', '@CookingChannel', '@tasty', '@nytfood', '@finecooking', '@mrcookingpanda'
# , '@FODMAPeveryday', '@FODMAPLife', '@FodmappedInfo', '@thefodmapdoctor', '@SimplyGlutenFre', '@gfliving', '@sibotest', '@manjulaskitchen', '@VegTimes', '@CookingLight', '@mealprepwl', '@thehealthygut', '@VitalGutHealth', '@pureguthealth', '@PaleoForBegin', '@PaleoLeap', '@ThePaleoMom', '@paleomagazine', '@PaleoHacks', '@paleogrubs',
# '@naturalgourmet', '@Low_Carb_Keto', '@NutritionTwins', '@mckelhill', '@WomensFitnessAu', '@WomensHealthMag', '@MensHealthMag', '@mjfit', '@thugkitchen', '@Leslie_Klenke', '@insidePN', '@ThisMamaCooks', '@EdibleWildFood', '@TheEarthDieter', '@HarvardHealth', '@EverydayHealth', '@DailyHealthTips']

#clean response from twitter

def clean_tweets(data, categories):
    topics = []
    for tweet in data:
        try:
            hashtag = tweet.entities['hashtags'][0]['text']
            tags = list(pd.DataFrame(tweet.entities['hashtags']).text)
            intersect = list(set(tags).intersection(categories))
            if len(intersect) > 0:
                hashtag = intersect[0]
            else:
                hashtag = hashtag
        except IndexError:
            words = set(re.sub("[^\w]", " ",  tweet.text).split())
            int2 = list(words.intersection(categories))
            if len(int2) > 0:
                hashtag = int2[0]
            else:
                hashtag = 'general'
        topics.append(hashtag)
    tweets = [{'title':tweet.id, 'date':tweet.created_at.date().strftime('%Y-%m-%d'),
       'description': tweet.text, 'source':tweet.user.screen_name,'source_id':tweet.user.id_str,
       'formality': 'Informal'
       ,'length': 1,'medium':'text'} for tweet in data]
    tweets = pd.DataFrame(tweets)
    tweets['param'] = topics
    return tweets



#CALL API
def twitter_api_call(list_handles, categories):
    empty = pd.DataFrame()
    for handle in list_handles:
        user_tweets = pd.DataFrame(clean_tweets(api.user_timeline(handle), categories))
        empty = empty.append(user_tweets, sort=True)
        print(handle)
    empty.title = empty.title.astype('str')
    empty['web_url'] = 'https://twitter.com/'+empty.source+'/status/'+empty.title
    empty['image_url'] = empty['web_url']
    return empty
import requests

class Tweet(object):
    def __init__(self, s, embed_str=False):
        if not embed_str:
            # Use Twitter's oEmbed API
            # https://dev.twitter.com/web/embedded-tweets
            api = 'https://publish.twitter.com/oembed?url={}'.format(s)
            response = requests.get(api)
            self.text = response.json()["html"]
        else:
            self.text = s

    def _repr_html_(self):
        return self.text


# Youtube 

In [103]:
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser
import pandas as pd
import pprint
import pafy
import warnings
warnings.filterwarnings("ignore")
from apikeys import *
from info import *


DEVELOPER_KEY = youtube_key
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"


def clean_youtube_time(string):
    if 'H' in string:
        minutes = int(string[string.find('H')+1:string.find('M')])
        hours = int(string[string.find('T')+1:string.find('H')]) * 60
        time = minutes + hours
    else:
        if 'M' in string:
            time = int(string[string.find('T')+1:string.find('M')])
        else:
            time = 1
    return time

def youtube_search(q, max_results=10,order="date", token=None, location=None, location_radius=None):

    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,developerKey=DEVELOPER_KEY)

    search_response = youtube.search().list(
    q=q,
    type="video",
    pageToken=token,
    order = order,
    part="id,snippet", # Part signifies the different types of data you want
    maxResults=max_results,
    location=location,
    locationRadius=location_radius).execute()

    all_dicts = []

    for search_result in search_response.get("items", []):
        if search_result["id"]["kind"] == "youtube#video":

            title = (search_result['snippet']['title'])

            videoId = (search_result['id']['videoId'])

            response = youtube.videos().list(
            part='statistics, snippet, contentDetails',
            id=search_result['id']['videoId']).execute()

            channelId = (response['items'][0]['snippet']['channelId'])
            channelTitle = (response['items'][0]['snippet']['channelTitle'])
            categoryId = (response['items'][0]['snippet']['categoryId'])
            favoriteCount = (response['items'][0]['statistics']['favoriteCount'])
            viewCount = (response['items'][0]['statistics']['viewCount'])
            date = pd.to_datetime((response['items'][0]['snippet']['publishedAt'])).date().strftime('%Y-%m-%d')
            description = response['items'][0]['snippet']['localized']['description']
            url = 'https://www.youtube.com/watch?v='+videoId
            image_url = response['items'][0]['snippet']['thumbnails']['default']['url']
            length = clean_youtube_time(response['items'][0]['contentDetails']['duration'])

        if 'commentCount' in response['items'][0]['statistics'].keys():
            commentCount = (response['items'][0]['statistics']['commentCount'])
        else:
            commentCount = []

        if 'tags' in response['items'][0]['snippet'].keys():
            tags = (response['items'][0]['snippet']['tags'])
        else:
            tags = []

        youtube_dict = {'tags':tags,'source_id': channelId,'source': channelTitle,'categoryId':categoryId,'title':title,'videoId':videoId,'viewCount':viewCount,'commentCount':commentCount,'favoriteCount':favoriteCount,
                        'formality':'Intermediate', 'medium':'video','date':date, 'description': description, 'web_url':url, 'image_url':image_url, 'length':length}
        all_dicts.append(youtube_dict)
    return pd.DataFrame(all_dicts)

def add_category(df, categories):
    # cats = ['keto','ketogenic','paleo','paleolithic','vegan','vegetarian']
    all_params = []
    for index, row in df.iterrows():
        try:
            intersect = list(set(row.tags).intersection(categories))
            if len(intersect) > 0:
                category = intersect[0]
            else:
                category = row.tags[0]
        except:
            category = 'none'
        all_params.append(category)
    df['param'] = all_params
    return df

def youtube_api_call(list_accounts, categories):
    empty_df = pd.DataFrame()
    errors = []
    for account in list_accounts:
        try:
            df = youtube_search(account)
            df = add_category(df, categories)
            empty_df = empty_df.append(df, sort=True)
            print(account)
        except:
            print(account + " EXCEPTION!!!!")
    return empty_df

# FB AND INSTA

In [10]:
# from instagram.client import InstagramAPI
# api = InstagramAPI(client_id="EAACl5okwUhQBAD6vhJiELgsatruedytMV67mvDuN2wgXEyRAXG7umE3T0KEweMhlWQWPgku5pF8KBwwOy9JJuFxx2chbYxPTjMdYA2mTd1DL6Jd5t4XwcetZBwhZC1Pyrl5MW2X90w6J9ZCZCWYlODApHzosU7ReUPZBmFITJj0cpsMZCDZATmEKip23ZA1nyA40Dv8IsLccLAZDZD")
# popular_media = api.media_popular(count=20)
# for media in popular_media:
#     print(media.images['standard_resolution'].url)

In [11]:
import facebook

graph = facebook.GraphAPI(access_token="EAACl5okwUhQBANPTECp6sKqxm3vubZCFmBBEtwdUQgLPAvAFLhpmk6ZAgDTB6Q6g6y6lvz3yjf7iZBW2ELCSrAwDhp28A1xse1QkQrvXowYgivMP0Rz3NRO8cEEy514LC7x3pAmFr1RBZBe7RJVVlsY70OWLHOgceSZCZC0HqzFdw13oEkEZC0Bm4ehPBxTlJFOoyiFE5f8ZCAZDZD", version="2.12")

# AUDIO

In [299]:
import requests
import json
import feedparser
import pandas as pd
import re
import time


def feed_urls(search_words, media_value='podcast', entity_value='podcast'):
#     Args:
#         search_words: The URL-encoded text string to be searched for
#         media_value: {movie, podcast, music, musicVideo, audiobook,
#                     shortFilm, tvShow, software, ebook, all} optional
#                     An optional variable, which indicates the media type to be searched for.
#         entity_value: Optional

    payload = {'term': search_words, 'media': media_value, 'entity' : entity_value}
    itunes_request = requests.get('https://itunes.apple.com/search', params=payload)
    itunes_result_json = itunes_request.json()
    result_count = itunes_result_json["resultCount"]
    if result_count > 0:
        feed_url = itunes_result_json["results"][0]['feedUrl']
    else:
        feed_url = "None"
    return feed_url

def fix_podcast_length(time):
    hours = int(time[0:2]) * 60
    minutes = int(time[4:5])
    length = hours + minutes
    return length

def df_podcast_episodes(feed_url):
    print(feed_url)
    if(len(feed_url) > 0):
        feed = feedparser.parse(feed_url)
        episodes = []
        for episode in feed.entries:
            info = dict()
            info['title'] = episode['title'] if 'title' in episode else ''
            info['description']= episode['summary'] if 'summary' in episode else ''
            info['length']= fix_podcast_length(episode['itunes_duration']) if 'itunes_duration' in episode else 1
            info['date']= pd.to_datetime(episode['published']).date().strftime('%Y-%m-%d') if 'published' in episode else ''
            info['medium'] = 'audio'
            info['formality'] = 'Intermediate'
            info['source'] = feed['feed']['title'] if 'title' in feed['feed'] else ''
            info['source_id'] = feed['feed']['title_detail']['base'] if 'title_detail' in feed['feed'] else ''
            try:
                info['web_url'] = episode['links'][0]['href'] if 'links' in episode else ''
            except:
                info['web_url'] = feed['feed']['title_detail']['base'] if 'title_detail' in feed['feed'] else ''
            info['image_url'] = feed['feed']['image']['href'] if 'image' in feed['feed'] else ''
            episodes.append(info)
        df = pd.DataFrame(episodes)
        return df
    else:
        print("No response!")

def add_category_to_audio(df, categories):
    all_params = []
    for index, row in df.iterrows():
        try:
            words = set(re.sub("[^\w]", " ",  row.description).split())
            intersect = list(words.intersection(categories))
            if len(intersect) > 0:
                category = intersect[0]
            else:
                category = 'general'
        except:
            category = 'general'
        all_params.append(category)
    df['param'] = all_params
    return df

def call_podcast_api(categories, podcasts):
    empty = pd.DataFrame()
    for podcast in podcasts:
        if podcast == 'None':
            pass
        else:
            try:
                url = feed_urls(podcast)
                df = df_podcast_episodes(url)
                df = add_category_to_audio(df, categories)
                empty = empty.append(df, sort=True)
                print('SUCCESS!')
                time.sleep(2)
            except:
                print('EXCEPTION')
                time.sleep(2)
    return empty


# EBOOKS

In [334]:
pd.set_option('display.max_columns', None)

In [368]:
def clean_ebook_date(df):
    dates = []
    for index, row in df.iterrows():
        date = pd.to_datetime(row.date).date().strftime('%Y-%m-%d')
        dates.append(date)
    df['date'] = dates
    return df


def ebook_search(search_word, media_value='ebook', entity_value='ebook'):
    payload = {'term': search_word, 'media': media_value, 'entity' : entity_value}
    itunes_request = requests.get('https://itunes.apple.com/search', params=payload)
    itunes_result_json = itunes_request.json()
    result_count = itunes_result_json["resultCount"]
    if result_count > 0:
        df = pd.DataFrame(itunes_result_json['results'])
        #NOTE LENGTH IS ACTUALLY THE PRICE BUT USE SAME LABEL FOR CONSISTENCY
        df = df.rename(index = str, columns= {'artistName': 'source','trackViewUrl':'web_url',
                                        'artworkUrl100':'image_url','price': 'length','releaseDate':'date','trackName':'title'})
        df['source_id'] = 'Itunes Ebook'
        df['formality'] = 'Formal'
        df['medium'] = 'text'
        df['param'] = search_word
        df = df.fillna(1)
        return clean_ebook_date(df)
    else:
        print('No Results!')
        return 'Empty'

def call_ebook_api(categories):
    empty = pd.DataFrame()
    for category in categories:
        df = ebook_search(category)
        if type(df) != str:
            empty = empty.append(df, sort=True)
            time.sleep(2)
            print('Added '+category)
        else:
            pass
    return empty


In [369]:
df = ebook_search('ketogenic')

In [370]:
test_cats = ['keto','ketogenic','paleo']
        

In [371]:
test = call_ebook_api(test_cats)

Added keto
Added ketogenic
Added paleo


In [372]:
test.isnull().sum()

artistId             0
artistIds            0
artistViewUrl        0
artworkUrl60         0
averageUserRating    0
currency             0
date                 0
description          0
fileSizeBytes        0
formality            0
formattedPrice       0
genreIds             0
genres               0
image_url            0
kind                 0
length               0
medium               0
param                0
source               0
source_id            0
title                0
trackCensoredName    0
trackId              0
userRatingCount      0
web_url              0
dtype: int64

In [373]:
test.dtypes

artistId               int64
artistIds             object
artistViewUrl         object
artworkUrl60          object
averageUserRating    float64
currency              object
date                  object
description           object
fileSizeBytes        float64
formality             object
formattedPrice        object
genreIds              object
genres                object
image_url             object
kind                  object
length               float64
medium                object
param                 object
source                object
source_id             object
title                 object
trackCensoredName     object
trackId                int64
userRatingCount      float64
web_url               object
dtype: object

In [374]:
test

,artistId,artistIds,artistViewUrl,artworkUrl60,averageUserRating,currency,date,description,fileSizeBytes,formality,formattedPrice,genreIds,genres,image_url,kind,length,medium,param,source,source_id,title,trackCensoredName,trackId,userRatingCount,web_url
0,427255804,[427255804],https://itunes.apple.com/us/artist/maria-emmerich/427255804?mt=11&uo=4,https://is1-ssl.mzstatic.com/image/thumb/Publication122/v4/57/5f/b0/575fb055-03d4-73ad-19cf-b967dcd20257/source/60x60bb.jpg,4.0,USD,2018-01-09,"Are you eating enough fat? Yes, fat. Despite what generations of health science has beaten into us during the last fifty or so years, humans thrive on high-fat, low-carb diets. Millions of people around the world have discovered that a ketogenic lifestyle is the key to weight loss, disease prevention and intervention, and a more vibrant life. Gone are the days when counting calories--or points!--is the path to better health, a slimmer waistline, relief from disease, and looking good naked.<br /><br />\nCraig and Maria Emmerich have partnered to write a book that goes well beyond the typical ketogenic recipes to create a book that digs deep into the science of ketogenic dieting, explains how dozens of diseases can be cured or controlled through ketogenic dieting, and how ketogenic diets...",18625890.0,Formal,$9.99,"[10069, 38, 9025, 11062, 9028, 10028]","[Health & Fitness, Books, Health, Mind & Body, Diet & Nutrition, Cookbooks, Food & Wine, Special Diet]",https://is1-ssl.mzstatic.com/image/thumb/Publication122/v4/57/5f/b0/575fb055-03d4-73ad-19cf-b967dcd20257/source/100x100bb.jpg,ebook,9.99,text,keto,Maria Emmerich,Itunes Ebook,Keto,Keto,1330497440,11.0,https://itunes.apple.com/us/book/keto/id1330497440?mt=11&uo=4
1,804478048,[804478048],https://itunes.apple.com/us/artist/suzanne-ryan/804478048?mt=11&uo=4,https://is5-ssl.mzstatic.com/image/thumb/Publication128/v4/bd/1d/3e/bd1d3e85-3ed5-51d3-7a8b-436eea1a90bd/source/60x60bb.jpg,5.0,USD,2017-12-12,"The ketogenic diet, a low-carb, high-fat way of eating, is remarkably effective at transforming people's lives, helping them shed pounds and find relief from common health conditions. No one knows this better than Suzanne Ryan. In her quest to overcome her lifelong struggle with her weight, she stumbled upon the ketogenic diet and decided to give it a shot. In just one year, she lost more than 100 pounds and reclaimed control over her health and well-being.<br /><br />\nSuzanne has shared every detail of her transformation, from the very first days of starting keto to her most recent successes, on her popular blog, Keto Karma, as well as on her YouTube channel and Instagram page. Her first book, ""Simply Keto"", isn't just a cookbook; it's a portal into Suzanne's life and dieting success...",48585704.0,Formal,$9.99,"[10069, 38, 9025, 9028, 10028]","[Health & Fitness, Books, Health, Mind & Body, Cookbooks, Food & Wine, Special Diet]",https://is5-ssl.mzstatic.com/image/thumb/Publication128/v4/bd/1d/3e/bd1d3e85-3ed5-51d3-7a8b-436eea1a90bd/source/100x100bb.jpg,ebook,9.99,text,keto,Suzanne Ryan,Itunes Ebook,Simply Keto,Simply Keto,1324366986,179.0,https://itunes.apple.com/us/book/simply-keto/id1324366986?mt=11&uo=4
2,996152816,[996152816],https://itunes.apple.com/us/artist/vincent-miles/996152816?mt=11&uo=4,https://is2-ssl.mzstatic.com/image/thumb/Publication7/v4/7b/ec/b8/7becb87d-9256-3c98-0f00-599caa503d36/source/60x60bb.jpg,4.5,USD,2015-05-17,"<b>Learn how you can lose weight, fight diabetes and feel great while on the Keto Diet!</b>&#xa0;<br /><br />The Ketogenic diet was one of the most popular diets of 2013 and also one of the most searched diets online as well. The Ketogenic diet is a&#xa0;<b>low-carb and moderate protein diet</b>&#xa0;that emphasizes eating mainly unprocessed fats such as nuts and olive oil.&#xa0;<br /><br /><b><i>I will teach you everything you need to know about the Keto Diet and how it will help you lose weight and get healthy.</i></b>&#xa0;<br /><br />The Keto Diet has been tout

In [ ]:
pd.to_datetime(episode['published']).date().strftime('%Y-%m-%d'

In [353]:
df['date'] = pd.to_datetime(df['date'])#.date()#.strftime('%Y-%m-%d')

In [347]:
df

,artistId,artistIds,source,web_url,image_url,artworkUrl60,averageUserRating,currency,description,fileSizeBytes,formattedPrice,genreIds,genres,kind,length,date,trackCensoredName,trackId,title,trackViewUrl,userRatingCount,source_id,formality,medium,param
0,1185137810,[1185137810],Louise Hendon,https://itunes.apple.com/us/artist/louise-hendon/1185137810?mt=11&uo=4,https://is5-ssl.mzstatic.com/image/thumb/Publication111/v4/e5/57/e5/e557e5c1-3a5a-60d4-37ca-9b59d4a1273f/source/100x100bb.jpg,https://is5-ssl.mzstatic.com/image/thumb/Publication111/v4/e5/57/e5/e557e5c1-3a5a-60d4-37ca-9b59d4a1273f/source/60x60bb.jpg,4.5,USD,"Easy-to-use ketogenic diet meal plan designed to make staying keto delicious and simple.&#xa0; Includes all the recipes with carb count and the meal plan is designed to be 20 grams of net carbohydrates or less daily.<br /><br />\nStarting a keto diet can be tough if you don't have a good plan for what to eat.&#xa0; That's where this 7-day keto meal plan comes in.&#xa0; I've done all the calculations, cooking, and planning for you.&#xa0; Now you can relax and enjoy all the benefits of a ketogenic diet stress-free.",3099645.0,Free,"[10028, 38, 9028, 9025, 10069]","[Special Diet, Books, Cookbooks, Food & Wine, Health, Mind & Body, Health & Fitness]",ebook,0.00,2016-12-09 08:00:00,7-Day Ketogenic Diet Meal Plan,1185137741,7-Day Ketogenic Diet Meal Plan,https://itunes.apple.com/us/book/7-day-ketogenic-diet-meal-plan/id1185137741?mt=11&uo=4,208.0,Itunes Ebook,Formal,text,ketogenic
1,1342849739,[1342849739],Emma Green,https://itunes.apple.com/us/artist/emma-green/1342849739?mt=11&uo=4,https://is2-ssl.mzstatic.com/image/thumb/Publication128/v4/16/a2/7f/16a27fc6-5c89-8b7b-1edd-e95ed58af72c/source/100x100bb.jpg,https://is2-ssl.mzstatic.com/image/thumb/Publication128/v4/16/a2/7f/16a27fc6-5c89-8b7b-1edd-e95ed58af72c/source/60x60bb.jpg,4.0,USD,"From the author of the bestselling title, ""How I Lost 100 Pounds!"" - you'll discover the simple secret to staying in a state of ketosis with 50 amazing top tasting recipes, included in this all-inclusive, special title.<br /><br />Eating better, losing real weight, and moving pounds on your waistline. Yes, no matter what your reasons, you are in for a surprisingly good treat. Giving you amazing, easy-to-follow recipes, and must-know information about the keto diet style and its necessary components. Yes, 50 Top Ketogenic Recipes is your must-have resource for starting and utilizing the ketogenic diet properly. Get into keto today and learn how good it can feel to lose weight and lead a brilliantly-healthy lifestyle using these must-have recipes.<br />The 50 Top Ketogenic Recipes cookbo...",748395.0,Free,"[10069, 38, 9025, 9028, 10028]","[Health & Fitness, Books, Health, Mind & Body, Cookbooks, Food & Wine, Special Diet]",ebook,0.00,2018-01-29 08:00:00,50 Top Ketogenic Recipes: Quick and Easy Keto Diet Recipes for Weight Loss and Optimum Health,1342916467,50 Top Ketogenic Recipes: Quick and Easy Keto Diet Recipes for Weight Loss and Optimum Health,https://itunes.apple.com/us/book/50-top-ketogenic-recipes-quick-easy-keto-diet-recipes/id1342916467?mt=11&uo=4,48.0,Itunes Ebook,Formal,text,ketogenic
2,1100322090,[1100322090],Nom Foodie,https://itunes.apple.com/us/artist/nom-foodie/1100322090?mt=11&uo=4,https://is4-ssl.mzstatic.com/image/thumb/Publication49/v4/26/a4/a7/26a4a73c-f7ac-9511-45d2-5e3361c94d1d/source/100x100bb.jpg,https://is4-ssl.mzstatic.com/image/thumb/Publication49/v4/26/a4/a7/26a4a73c-f7ac-9511-45d2-5e3361c94d1d/source/60x60bb.jpg,4.5,USD,"Snacks, desserts, keto fuel? This ketogenic cookbook has you covered!<br /><br />Fat bombs are delicious high fat, low carb, ketogenic snacks designed to keep you fueled throughout the day. In this keto cookbook, I've got 20 delicious paleo ketogenic fat bomb recipes that you'll love. From Blueberry Nut Bars to Bacon Avocado Balls and Cinnamon Walnut Cashew Cheese Sandwiches. I'm sure you'll find a recipe to satisfy your keto diet.<br /><br 

In [ ]:
def clean_ebook(search_results):
    

In [323]:
test = feed_urls_ebook('ketogenic', media_value='ebook', entity_value='ebook')

In [324]:
test

{'resultCount': 50,
 'results': [{'artworkUrl60': 'https://is5-ssl.mzstatic.com/image/thumb/Publication111/v4/e5/57/e5/e557e5c1-3a5a-60d4-37ca-9b59d4a1273f/source/60x60bb.jpg',
   'artworkUrl100': 'https://is5-ssl.mzstatic.com/image/thumb/Publication111/v4/e5/57/e5/e557e5c1-3a5a-60d4-37ca-9b59d4a1273f/source/100x100bb.jpg',
   'artistViewUrl': 'https://itunes.apple.com/us/artist/louise-hendon/1185137810?mt=11&uo=4',
   'trackCensoredName': '7-Day Ketogenic Diet Meal Plan',
   'fileSizeBytes': 3099645,
   'trackViewUrl': 'https://itunes.apple.com/us/book/7-day-ketogenic-diet-meal-plan/id1185137741?mt=11&uo=4',
   'trackId': 1185137741,
   'trackName': '7-Day Ketogenic Diet Meal Plan',
   'formattedPrice': 'Free',
   'artistIds': [1185137810],
   'genreIds': ['10028', '38', '9028', '9025', '10069'],
   'releaseDate': '2016-12-09T08:00:00Z',
   'currency': 'USD',
   'description': "Easy-to-use ketogenic diet meal plan designed to make staying keto delicious and simple.&#xa0; Includes all 

In [325]:
test['results'][0].keys()

dict_keys(['artworkUrl60', 'artworkUrl100', 'artistViewUrl', 'trackCensoredName', 'fileSizeBytes', 'trackViewUrl', 'trackId', 'trackName', 'formattedPrice', 'artistIds', 'genreIds', 'releaseDate', 'currency', 'description', 'artistId', 'artistName', 'genres', 'kind', 'price', 'averageUserRating', 'userRatingCount'])

In [328]:
test['results'][0]

{'artworkUrl60': 'https://is5-ssl.mzstatic.com/image/thumb/Publication111/v4/e5/57/e5/e557e5c1-3a5a-60d4-37ca-9b59d4a1273f/source/60x60bb.jpg',
 'artworkUrl100': 'https://is5-ssl.mzstatic.com/image/thumb/Publication111/v4/e5/57/e5/e557e5c1-3a5a-60d4-37ca-9b59d4a1273f/source/100x100bb.jpg',
 'artistViewUrl': 'https://itunes.apple.com/us/artist/louise-hendon/1185137810?mt=11&uo=4',
 'trackCensoredName': '7-Day Ketogenic Diet Meal Plan',
 'fileSizeBytes': 3099645,
 'trackViewUrl': 'https://itunes.apple.com/us/book/7-day-ketogenic-diet-meal-plan/id1185137741?mt=11&uo=4',
 'trackId': 1185137741,
 'trackName': '7-Day Ketogenic Diet Meal Plan',
 'formattedPrice': 'Free',
 'artistIds': [1185137810],
 'genreIds': ['10028', '38', '9028', '9025', '10069'],
 'releaseDate': '2016-12-09T08:00:00Z',
 'currency': 'USD',
 'description': "Easy-to-use ketogenic diet meal plan designed to make staying keto delicious and simple.&#xa0; Includes all the recipes with carb count and the meal plan is designed t